<a href="https://colab.research.google.com/github/ricyoung/23andme2vcf/blob/master/hw4_tf2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds


In [0]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised=True)



local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [0]:
print(mnist_info) 

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [0]:
# one hot encode for 10 MNIST classes
def my_one_hot(feature, label):
    return feature, tf.one_hot(label, depth=10)

# load your data from tfds
mnist_train, train_info = tfds.load(name="mnist", with_info=True, as_supervised=True, split=tfds.Split.TRAIN)

# convert your labels in one-hot
mnist_train = mnist_train.map(my_one_hot)

# you can batch your data here
# mnist_train = mnist_train.batch(8)

In [0]:
# batch_size=-1 to get the full dataset in NumPy arrays from the returned tf.Tensor object


mnist_train = tfds.load(name="mnist", split=tfds.Split.TRAIN, batch_size=-1 ) 
mnist_test = tfds.load(name="mnist", split=tfds.Split.TEST, batch_size=-1)

In [0]:


# tfds.as_numpy return a generator that yields NumPy array records out of a tf.data.Dataset
mnist_train = tfds.as_numpy(mnist_train) 
mnist_test = tfds.as_numpy(mnist_test)





In [0]:
x_train, y_train = mnist_train["image"], mnist_train["label"] # seperate the x and y
x_test, y_test = mnist_test["image"], mnist_test["label"]

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score

In [0]:
# kf = KFold(n_splits=10, shuffle=True )
# kf.get_n_splits(x_train)

10

In [0]:
# print(kf)

KFold(n_splits=10, random_state=None, shuffle=True)


In [0]:
# This doesn't work 

# for train_index, test_index in kf.split(x_train):
#   print("TRAIN:", train_index, "TEST:", test_index) 
  # X_train, X_test =x_train[train_index], x_train[test_index]


TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [    5    25    29 ... 59971 59976 59987]
TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [    3    14    18 ... 59968 59992 59993]
TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [   11    12    13 ... 59983 59984 59988]
TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [    7    36    37 ... 59961 59975 59985]
TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [    9    22    23 ... 59965 59980 59982]
TRAIN: [    3     4     5 ... 59996 59997 59998] TEST: [    0     1     2 ... 59956 59995 59999]
TRAIN: [    0     1     2 ... 59995 59998 59999] TEST: [    6     8    20 ... 59986 59996 59997]
TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [   19    30    33 ... 59966 59974 59991]
TRAIN: [    0     1     2 ... 59997 59998 59999] TEST: [    4    24    31 ... 59978 59989 59994]
TRAIN: [    0     1     2 ... 59996 59997 59999] TEST: [   21    27    42 ... 59973 59990 59998]


In [0]:
#k-fold - split data index into 10parts
batch = []
for i in range (10):
    batch.append([])
    for j in range (len(x_train)):
        if j%10 == i:
            batch[i].append(x_train[j])


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>


In [0]:
# for train_index, test_index in kf.split(x_train):
for i in range(len(batch)):
    #reduces to 0-1 from 0-255
    #this may not have much of a change on the output
    x_train = tf.keras.utils.normalize(x_train, axis=1)
    x_test = tf.keras.utils.normalize(x_test, axis=1)

    #Using sequential model with 2 layered neural network with each having 128 neurons
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

    model.compile(optimizer= 'adam',
                  
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=6)

    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(i)
    print(val_acc, val_loss)

Epoch 1/6
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2099 - accuracy: 0.9368
Epoch 2/6
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0869 - accuracy: 0.9731
Epoch 3/6
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0597 - accuracy: 0.9815
Epoch 4/6
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0439 - accuracy: 0.9855
Epoch 5/6
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0356 - accuracy: 0.9883
Epoch 6/6
313/313 [==============================] - 1s 2ms/step - loss: 0.1066 - accuracy: 0.9727
0
0.9726999998092651 0.10661064833402634
Epoch 1/6
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2111 - accuracy: 0.9353
Epoch 2/6
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0878 - accuracy: 0.9725
Epoch 3/6
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0590 - accuracy: 0.9813
Epoch 4/6
1875/1875 [=======================